In [40]:
from tilepack.builder import build_tile_packages
from configparser import ConfigParser
import os
import shutil
import secrets as secrets
from glob import glob, iglob
from pathlib import Path
import zipfile
import json

os.environ["MAPZEN_API_KEY"] = secrets.MAPZEN_API_KEY

# Download MBTiles & GeoJSON by script

In [2]:
cfg: ConfigParser = ConfigParser()
cfg.read('../config/config.cfg')
cfg_dict: dict = cfg._sections['mbtiles_downloader']
cfg_dict = {
    'min_lon': float(cfg_dict['min_lon']),
    'min_lat': float(cfg_dict['min_lat']),
    'max_lon': float(cfg_dict['max_lon']),
    'max_lat': float(cfg_dict['max_lat']),
    'min_zoom': int(cfg_dict['min_zoom']),
    'max_zoom': int(cfg_dict['max_zoom']),
    'output': cfg_dict['output']
}
cfg_dict

{'min_lon': 14.397,
 'min_lat': 50.1133,
 'max_lon': 14.4137,
 'max_lat': 50.1224,
 'min_zoom': 10,
 'max_zoom': 16,
 'output': 'zoo_prague_map'}

In [3]:
mapzen_url_prefix = os.getenv('MAPZEN_URL_PREFIX', 'https://tile.nextzen.org/tilezen')
mapzen_api_key = os.getenv('MAPZEN_API_KEY', None)
if(mapzen_api_key is None):
    raise RuntimeError('Environmental variable MAPZEN_API_KEY is not set.')

args = {
    'tile_size': 512,
    'tile_format': 'json',
    'output_formats': ['mbtiles', 'zipfile'],
    'layer': 'all',
    'type': 'vector',
    'tile_compression': False,
    'concurrency': 1,
    'api_key': mapzen_api_key,
    'url_prefix': mapzen_url_prefix
}
args |= cfg_dict

# Get MBTiles & GeoJSON
build_tile_packages(**args)

# Move created files into a folder
folder_path = Path(args['output'])
if os.path.isdir(folder_path):
    shutil.rmtree(folder_path)
    
os.makedirs(folder_path)
for file in glob(f'{folder_path}*.*'):
    shutil.move(file, folder_path)
    
# Unzip GeoJSON files
geojson_path = folder_path / 'geojsons'
with zipfile.ZipFile(folder_path / f'{folder_path}.zip', 'r') as zip_ref:
    zip_ref.extractall(geojson_path)

30 good, 0 errored of 30 (100.00%) tiles for zoo_prague_map


In [39]:
# Get data from GeoJSONs
for geojson in iglob(str(geojson_path / 'all' / '**/*.json'), recursive=True):
    with open(geojson) as f:
        data = json.load(f)
        for poi in data['pois']['features']:
            poi = poi['properties']
            if(poi['kind'] == 'animal'):
                res: dict = {
                    '_id': poi['id'],
                    'name': poi['name']
                }
                print(res)

{'_id': 4636225666, 'name': 'žirafy, pštrosi, přímorožci, antilopy losí'}
{'_id': 4639672095, 'name': 'giboni'}
{'_id': 4639672096, 'name': 'plameňáci'}
{'_id': 4639672111, 'name': 'tamaríni'}
{'_id': 4639672112, 'name': 'gepardi'}
{'_id': 4639672113, 'name': 'kotulové'}
{'_id': 4639672115, 'name': 'kolpíci'}
{'_id': 4639672116, 'name': 'pelikáni'}
{'_id': 4639672117, 'name': 'mravenečníci'}
{'_id': 4639672118, 'name': 'sitatungy'}
{'_id': 4639672142, 'name': 'ibisové'}
{'_id': 4639672143, 'name': 'paovce'}
{'_id': 4639672145, 'name': 'tygři'}
{'_id': 4791797957, 'name': 'gorily'}
{'_id': 4791798031, 'name': 'lvi'}
{'_id': 4791798037, 'name': 'želvy'}
{'_id': 4636225657, 'name': 'tygři'}
{'_id': 4636225658, 'name': 'sovy'}
{'_id': 4636225667, 'name': 'vodušky'}
{'_id': 4636225668, 'name': 'surikaty'}
{'_id': 4636225677, 'name': 'hrabáči'}
{'_id': 4636225678, 'name': 'štětkouni'}
{'_id': 4636225685, 'name': 'antilopy'}
{'_id': 4636225686, 'name': 'jihoamerické šelmy'}
{'_id': 4636225687